<a href="https://colab.research.google.com/github/adilkhan17/mlcourse.ai/blob/master/HW2_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: Токтаганов Адиль

Группа: DS-11

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [9]:
pip install osmapi

  Created wheel for osmapi: filename=osmapi-1.3.0-cp36-none-any.whl size=13228 sha256=aca7c06cf2f7dcda9f2597be831254a8a446930ccd25022ce88371a7cde305c1
  Stored in directory: /root/.cache/pip/wheels/2c/ce/bb/177779128309a003ebeb34cc3b9464ec9111b4cd7886a61933
Successfully built osmapi


In [10]:
pip install geopandas

     |████████████████████████████████| 972kB 2.8MB/s 
     |████████████████████████████████| 14.8MB 276kB/s 


In [11]:
pip install overpy

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for overpy: filename=overpy-0.4-cp36-none-any.whl size=45772 sha256=6f50c6f91188a4c1552b8c74c0cf6715d4dc4c8afcb5a182e9d3a0aeef67402f
  Stored in directory: /root/.cache/pip/wheels/10/53/c2/e6b6f97e7bb419193bd3aafbe38628666f0f93a1cec9dc521d
Successfully built overpy


In [12]:
pip install OSMPythonTools

     |████████████████████████████████| 184kB 3.7MB/s 
  Created wheel for OSMPythonTools: filename=OSMPythonTools-0.2.9-cp36-none-any.whl size=12770 sha256=683ccf32b7066ffc0aa39f62ed5aa85ec6a807d1750946503f294fd40ef1855d
  Stored in directory: /root/.cache/pip/wheels/4e/65/34/afe0b46e3dbb1b3da010967781d4999084bfe5c63d7b2eeefd
  Created wheel for pytest-sugar: filename=pytest_sugar-0.9.4-py2.py3-none-any.whl size=8972 sha256=41c934aa2b229f0496e2a0579173f3611676f904a0897c23efe88147606d6377
  Stored in directory: /root/.cache/pip/wheels/72/ac/7f/55050ee786fa4a2bb2d05dea0696eab826ff1d5b8a7dbd5883
Successfully built OSMPythonTools pytest-sugar


In [13]:
!sudo apt install python3-rtree
!pip install "rtree>=0.8,<0.9"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources python3-rtree
0 upgraded, 5 newly installed, 0 to remove and 11 not upgraded.
Need to get 671 kB of archives.
After this operation, 3,948 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-pkg-resources all 39.0.1-2 [98.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 k

In [14]:
import osmapi as osm
from PIL import Image
import requests
from io import BytesIO
import geopandas as gpd
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.api import Api
from shapely.geometry import Point, LineString, MultiPoint
import folium
import random
from folium.plugins import TimestampedGeoJson
from datetime import datetime, timedelta
import numpy as np
import math
import rtree.index

def distance_haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)

    a = math.sin(dLat/2.0) * math.sin(dLat/2.0) + math.sin(dLon/2.0) * math.sin(dLon/2.0) * math.cos(lat1) * math.cos(lat2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    
    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1)*math.cos(lat2)*math.cos(dLon)
    brng = math.atan2(y,x)
    return d, (brng/(2*math.pi))*360

In [15]:
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
afghanistan_gdf = world.loc[world['name'].str.contains('Afghanistan')]
afghanistan_gdf.crs = "EPSG:4326"
afghanistan_geometry = afghanistan_gdf.geometry.ravel()[0]
afghanistan_center = (afghanistan_geometry.centroid.y, afghanistan_geometry.centroid.x)

In [16]:
map = folium.Map(afghanistan_center, zoom_start=2)
folium.GeoJson(afghanistan_gdf).add_to(map)
folium.Marker(
    location=afghanistan_center,
    tooltip='Afghanistan Center',
    icon=folium.Icon(color='green')
).add_to(map)

def point_distance(tuple_point1, tuple_point2):
    return Point(tuple_point1[1], tuple_point1[0]).distance(Point(tuple_point2[1], tuple_point2[0]))

def random_search(seed = 112):
    MIN_LAT, MAX_LAT, MIN_LON, MAX_LON = -90, 90, -180, 180

    random.seed(seed)
    startPoint = (random.randrange(MIN_LAT, MAX_LAT + 1), random.randrange(MIN_LON, MAX_LON + 1))

    currentPoint = startPoint                                                       
    currentDistance = point_distance(startPoint, afghanistan_center)
    
    i = 0   
    points = [startPoint]
    while not afghanistan_geometry.contains(Point(currentPoint[1], currentPoint[0])):
        add = 5
        down, up, left, right = (currentPoint[0] - add, currentPoint[1]), (currentPoint[0] + add, currentPoint[1]), (currentPoint[0], currentPoint[1] - add), (currentPoint[0], currentPoint[1] + add)
        wasSet = False
        while (True):
            for point in [down, up, left, right]:
                dist = point_distance(point, afghanistan_center)
                if dist < currentDistance:
                    currentDistance = dist
                    currentPoint =  (random.randrange(point[0] - 2, point[0] + 2), random.randrange(point[1] - 2, point[1] + 2)) 
                    wasSet = True
            if wasSet or afghanistan_geometry.contains(Point(currentPoint[1], currentPoint[0])):
                break

            add /= 2
            down, up, left, right = (currentPoint[0] - add, currentPoint[1]), (currentPoint[0] + add, currentPoint[1]), (currentPoint[0], currentPoint[1] - add), (currentPoint[0], currentPoint[1] + add)
        i += 1
        points.append(currentPoint)
    return points
points = random_search(5)

def create_geojson_features(points):
    features = []
    
    for i, p in enumerate(points):
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[p[1], p[0]]
            },
            'properties': {
                'time': (datetime.today() + i * timedelta(hours=0, minutes=1)).strftime("%Y-%m-%d %H:%M:%S"),
                'style': {'color' : ''},
                'icon': '100',
                'iconstyle':{
                    'fillColor': '#00FF00',
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 10000
                }
            }
        }
        features.append(feature)
    return features

json = create_geojson_features(points)
TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': json},
    period='PT1M',
    duration='PT5M',
    auto_play=True,
    loop=False,
    loop_button=False
).add_to(map)

map

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [17]:
import overpy
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
import pandas as pd

In [18]:
pip install country-list

     |████████████████████████████████| 1.5MB 2.7MB/s 


In [45]:
from country_list import countries_for_language
countries = tuple(countries_for_language('en'))

cntr = []
codes = []
nominatim = Nominatim()
for i in countries:
  area = nominatim.query(i[1]).areaId()
  try:
    if area % 10 == 0:
      codes.append(i[0])
      cntr.append(i[1])
  except TypeError:
    pass

target_cntr = pd.DataFrame()
target_cntr['country_name'] = cntr
target_cntr['country_code'] = codes

In [46]:
print(target_cntr.shape)
target_cntr.head()

(28, 2)


,country_name,country_code
0,Åland Islands,AX
1,Antigua & Barbuda,AG
2,Australia,AU
3,Azerbaijan,AZ
4,Bangladesh,BD


In [ ]:
houses_cnt = []

for i in target_cntr.country_code:
  try:
    r = api.query("""
        area["ISO3166-1"={}][admin_level=2];
        (node["building"~"apartments|house"](area);
        way["building"~"apartments|house"](area);
        rel["building"="apartments"](area);
        );
        out center;
    """.format(i))

    coords  = []
    coords += [(float(node.lon), float(node.lat)) 
              for node in r.nodes]
    coords += [(float(way.center_lon), float(way.center_lat)) 
              for way in r.ways]
    coords += [(float(rel.center_lon), float(rel.center_lat)) 
              for rel in r.relations]

    cnt = len(coords)
    houses_cnt.append(cnt)
  except:
    print('Error on cntry: ', i)

In [24]:
api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "BR"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

BR_cnt = len(coords)

In [25]:
api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "BQ"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

BQ_cnt = len(coords)

In [26]:
api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "ST"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

ST_cnt = len(coords)

In [27]:
api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "TK"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

TK_cnt = len(coords)

In [28]:
api = overpy.Overpass()

r = api.query("""
area["ISO3166-1"= "UZ"][admin_level=2];
(node["building"~"apartments|house"](area);
 way["building"~"apartments|house"](area);
 rel["building"="apartments"](area);
);
out center;
""")
coords = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

UZ_cnt = len(coords)

In [33]:
target_cntr = target_cntr[target_cntr['country_code'].isin(['BR','BQ','ST','TK','UZ'])]

In [30]:
houses_cnt_tst = houses_cnt.copy()

houses_cnt_tst.insert(5, BR_cnt)
houses_cnt_tst.insert(8, BQ_cnt)
houses_cnt_tst.insert(22, ST_cnt)
houses_cnt_tst.insert(26, TK_cnt)
houses_cnt_tst.insert(27, UZ_cnt)

In [34]:
target_cntr['country_houses_cnt'] = houses_cnt_tst

In [35]:
target_cntr.sort_values(by = ['country_houses_cnt'], ascending = False).head(1)

,country_name,country_code,country_houses_cnt
5,Brazil,BR,8005


In [36]:
api = overpy.Overpass()
r = api.query("""
area["ISO3166-1"="BR"][admin_level=2];
(node["amenity"="atm"](area);
 way["amenity"="atm"](area);
 rel["amenity"="atm"](area);
);
out center;
""")
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

In [38]:
from rtree import index
from rtree.index import Rtree

r_tree = Rtree()

for i, j in enumerate(coords):
    r_tree.insert(i, j+j, j)
    
five_atm = list(r_tree.nearest([afghanistan_geometry.centroid.x, afghanistan_geometry.centroid.y], 5, objects = 'raw'))

In [39]:
import math

def haversine(coord1, coord2):
    R = 6_372.8  # Earth radius in km
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [40]:
sum = 0
for i in five_atm:
    sum += haversine(i, (afghanistan_geometry.centroid.x, afghanistan_geometry.centroid.y))

sum

57100.118089386626

2 *subtask* Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице

In [47]:
api = overpy.Overpass()
pharm_cnt = []

for i in target_cntr.country_code:
  try:
    r = api.query("""
        area["ISO3166-1"={}][admin_level=2];
        (node["amenity" = "pharmacy"](area);
        );
        out center; """.format(i))

    coords  = []
    coords += [(float(node.lon), float(node.lat)) 
              for node in r.nodes]
    coords += [(float(way.center_lon), float(way.center_lat)) 
              for way in r.ways]
    coords += [(float(rel.center_lon), float(rel.center_lat)) 
              for rel in r.relations]

    cnt = len(coords)
    pharm_cnt.append(cnt)
  except:
    print('Error on cntry: ', i)

In [49]:
target_cntr['pharm_cnt'] = pharm_cnt

In [50]:
target_cntr.sort_values(by = ['pharm_cnt'], ascending=False).head(5)

,country_name,country_code,pharm_cnt
5,Brazil,BR,6798
21,Portugal,PT,2113
27,Uzbekistan,UZ,1931
2,Australia,AU,1844
25,Tanzania,TZ,1814


In [51]:
api = overpy.Overpass()
r = api.query("""
area["ISO3166-1"="BR"][admin_level=2];
(node["amenity"="school"](area);
 way["amenity"="school"](area);
 rel["amenity"="school"](area);
);
out center;
""")
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

In [52]:
r_tree = Rtree()

for i, j in enumerate(coords):
    r_tree.insert(i, j+j, j)
    
five_schools = list(r_tree.nearest([afghanistan_geometry.centroid.x, afghanistan_geometry.centroid.y], 5, objects = 'raw'))

In [53]:
sum = 0
for i in five_schools:
    sum += haversine(i, (afghanistan_geometry.centroid.x, afghanistan_geometry.centroid.y))

sum

58823.17869097722

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [5]:
import json
import requests
from tqdm import tqdm

import folium
import pyproj
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

from openrouteservice import client

import warnings
warnings.filterwarnings("ignore")

In [2]:
pip install pyproj

     |████████████████████████████████| 6.5MB 2.5MB/s 


In [4]:
pip install openrouteservice

In [6]:
url = 'https://data.cityofnewyork.us/api/views/a3nt-yts4/rows.json?accessType=DOWNLOAD'

def CreateBufferPolygon(point_in, resolution=10, radius=10):    

    sr_wgs = pyproj.Proj(init='epsg:4326')
    sr_utm = pyproj.Proj(init='epsg:32632') # WGS84 UTM32N
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in) # unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 10 m buffer
    
    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84

    return poly_wgs

In [7]:
api_key = '5b3ce3597851110001cf62483414bd8dcf43473e9b60ea40e4aa0533' # Individual api key
clnt = client.Client(key=api_key) # Create client with api key
rostock_json = requests.get(url).json() # Get data as JSON

map_params = {'tiles':'Stamen Toner',
              'location':([40.769361, -73.977655]),
              'zoom_start': 12}
map1 = folium.Map(**map_params)

# Populate a construction site buffer polygon list
sites_poly = []

for item in tqdm(rostock_json['data']):
    coords = item[8]
    coords = coords.replace('POINT (','').replace(')','')
    
    # 1992 школ не влазили в фолиум, уменшим областью чуть шире заданных начала и конца пути
    site_coords = [float(x) for x in coords.split(' ')]

    if site_coords[0] < -73.96 and site_coords[0] > -73.99 and site_coords[1] > 40.70 and site_coords[0] < 40.79:

        folium.features.Marker(list(reversed(site_coords)),
                                   popup='School<br>{0}'.format(site_coords)).add_to(map1)

        # Create buffer polygons around construction sites with 10 m radius and low resolution
        site_poly_coords = CreateBufferPolygon(site_coords,
                                                   resolution=2, # low resolution to keep polygons lean
                                                   radius=10)
        sites_poly.append(site_poly_coords)

        site_poly_coords = [(y,x) for x,y in site_poly_coords] # Reverse coords for folium/Leaflet
        folium.vector_layers.Polygon(locations=site_poly_coords,
                                          color='#ffd699',
                                          fill_color='#ffd699',
                                          fill_opacity=0.2,
                                          weight=3).add_to(map1)
    
map1

100%|██████████| 1992/1992 [01:27<00:00, 22.85it/s]


In [8]:
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=0.5)

# Create new map to start from scratch
map_params.update({'location': ([40.769361, -73.977655]),
                   'zoom_start': 13})
map2 = folium.Map(**map_params)

# Request normal route between appropriate locations without construction sites
request_params = {'coordinates': [[-73.977655, 40.769361],
                                 [-73.98806, 40.70306]],
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': 'false',}
route_normal = clnt.directions(**request_params)
folium.features.GeoJson(data=route_normal,
                        name='Route without Schools',
                        style_function=style_function('#FF0000'),
                        overlay=True).add_to(map2)

# Buffer route with 0.009 degrees (really, just too lazy to project again...)
route_buffer = LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.009)
folium.features.GeoJson(data=geometry.mapping(route_buffer),
                        name='Route Buffer',
                        style_function=style_function('#FFFF00'),
                        overlay=True).add_to(map2)

# Plot which construction sites fall into the buffer Polygon
sites_buffer_poly = []
for site_poly in sites_poly:
    poly = Polygon(site_poly)
    if route_buffer.intersects(poly):
        folium.features.Marker(list(reversed(poly.centroid.coords[0]))).add_to(map2)
        sites_buffer_poly.append(poly)

map2